## Handwritten Image Detection with Keras using MNIST data

In this exercise we will work with image data: specifically the famous MNIST data set.  This data set contains 70,000 images of handwritten digits in grayscale (0=black, 255 = white), 60,000 for training and 10,000 for testing.  The images are 28 pixels by 28 pixels for a total of 784 pixels.  This is quite small by image standards.  Also, the images are well centered and isolated.  This makes this problem solvable with standard fully connected neural nets without too much pre-work.

In the first part of this notebook, we will walk you through loading in the data, building a fully connected network, and training it.  Then it will be your turn to implement LeNet-5, try other different models and see if you can improve performance

In [0]:
# Preliminaries

from __future__ import print_function

import keras
from keras.datasets import mnist # easy loading of MNIST
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Let's explore the dataset a little bit

In [0]:
# Load the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
x_train[0].shape

In [0]:
#Let's just look at a particular example to see what is inside

x_train[333]  ## Just a 28 x 28 numpy array of ints from 0 to 255

In [0]:
# What is the corresponding label in the training set?
y_train[333]

In [0]:
# Let's see what this image actually looks like

plt.imshow(x_train[333], cmap='Greys_r')

In [0]:
# this is the shape of the np.array x_train
# it is 3 dimensional.
print(x_train.shape, 'train samples')
print(x_test.shape, 'test samples')

In [0]:
## For our purposes, these images are just a vector of 784 inputs, so let's convert
fc_x_train = x_train.reshape(len(x_train), 28*28)
fc_x_test = x_test.reshape(len(x_test), 28*28)

## Keras works with floats, so we must cast the numbers to floats
fc_x_train = fc_x_train.astype('float32')
fc_x_test = fc_x_test.astype('float32')

## Normalize the inputs so they are between 0 and 1
fc_x_train /= 255
fc_x_test /= 255


In [0]:
# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

y_train[333]  # now the digit k is represented by a 1 in the kth entry (0-indexed) of the length 10 vector

In [0]:
# We will build a model with two hidden layers of size 512
# Fully connected inputs at each layer
# We will use dropout of .2 to help regularize
model_1 = Sequential()
model_1.add(Dense(64, activation='relu', input_shape=(784,)))
model_1.add(Dropout(0.2))
model_1.add(Dense(64, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(10, activation='softmax'))

In [0]:
## Note that this model has a LOT of parameters
model_1.summary()

In [0]:
# Let's compile the model
learning_rate = .001
model_1.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=learning_rate),
              metrics=['accuracy'])
# note that `categorical cross entropy` is the natural generalization 
# of the loss function we had in binary classification case, to multi class case

In [0]:
# And now let's fit.

batch_size = 128  # mini-batch with 128 examples
epochs = 30
history_1 = model_1.fit(
    fc_x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(fc_x_test, y_test)) # this for plotting the loss curve

In [0]:
## We will use Keras evaluate function to evaluate performance on the test set

score_1 = model_1.evaluate(fc_x_test, y_test, verbose=0)
print('Test loss:', score_1[0])
print('Test accuracy:', score_1[1])

In [0]:
def plot_loss_accuracy(history):
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 2, 1)
    ax.plot(history.history["loss"],'r-x', label="Train Loss")
    ax.plot(history.history["val_loss"],'b-x', label="Validation Loss")
    ax.legend()
    ax.set_title('cross_entropy loss')
    ax.grid(True)


    ax = fig.add_subplot(1, 2, 2)
    ax.plot(history.history["acc"],'r-x', label="Train Accuracy")
    ax.plot(history.history["val_acc"],'b-x', label="Validation Accuracy")
    ax.legend()
    ax.set_title('accuracy')
    ax.grid(True)
    

plot_loss_accuracy(history_1)

This is reasonably good performance, but we can do even better!  Next you will build an even bigger network and compare the performance.

## Exercise
### Your Turn: Build your own model
Use the Keras "Sequential" functionality to build `model_2` with the following specifications:

1. Two hidden layers.
2. First hidden layer of size 400 and second of size 300
3. Dropout of .4 at each layer
4. How many parameters does your model have?  How does it compare with the previous model?
4. Train this model for 20 epochs with RMSProp at a learning rate of .001 and a batch size of 128




In [0]:
### Build your model here


### Print its summary

batch_size = 
epochs = 
learning_rate = 


### Compile it. Remember to set optimizer and learning rate


### Fit and Evaluate the model
print('')
print('Test loss:', ? )
print('Test accuracy:', ?)

In [0]:
plot_loss_accuracy(history_2)

### Your Turn: Build your own model
Implement LeNet-5 in `model_3`.

How many parameters does your model have?  How does it compare with the previous model?

In [0]:
### Build your model here
from keras.layers import Conv2D, AveragePooling2D, Flatten


### Print its summary


In [0]:
## prepare the images
# Conv2D accept input of the form (batch, rows, cols, channels) -> we need to add a new column to x_train and x_test
# See https://keras.io/layers/convolutional/ for more details
# Reshape the training and test set
ln_x_train = x_train.reshape(x_train.shape[0], 28, 28, 1) # <- the reshape is necessary as Conv2D layer takes as input examples that are a matrix with a cell for each pixel and each pixel is an array 
ln_x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)    #    containing information about each channel of the image. So, wih length 1 for greyscale, 3 for RGB, 4 for CMYK, etc.

# Padding the images by 2 pixels since in the paper input images were 32x32
ln_x_train = np.pad(ln_x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
#                             padding at the beginning and end of the first dimension -> 60000
#                                   padding for the second dimension (we are here in the matrix) -> 2 pixels at the beginning and at the end of each row
#                                          padding for the second dimension (we are here in the matrix) -> 2 pixels at the beginning and at the end of each column
#                                                   padding for the number of channel
ln_x_test = np.pad(ln_x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

## Keras works with floats, so we must cast the numbers to floats
ln_x_train = ln_x_train.astype('float32')
ln_x_test = ln_x_test.astype('float32')

## Normalize the inputs so they are between 0 and 1
ln_x_train /= 255
ln_x_test /= 255

# this is the shape of the np.array ln_x_train
# it is 3 dimensional.
print(ln_x_train.shape, 'train samples')
print(ln_x_test.shape, 'test samples')

#Visualizing the data
plt.imshow(ln_x_train[333,:,:,0], cmap='Greys_r')

In [0]:
model_3.compile(loss='categorical_crossentropy',
              optimizer='adam',  # use of Adam here
              metrics=['accuracy'])

In [0]:
batch_size = 128
epochs = 10


history_3 = model_3.fit(ln_x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(ln_x_test, y_test))

score_3 = model_3.evaluate(ln_x_test, y_test, verbose=0)
print('')
print('Test loss:', score_3[0])
print('Test accuracy:', score_3[1])


In [0]:
plot_loss_accuracy(history_3)

## SOLUTION

## Think about the following questions

1) How do `model_1`, `model_2` and `model_3` compare?  Which do you prefer?  If you were going to put one into production, which would you choose and why?

2) Compare the trajectories of the loss function on the training set and test set for each model?  How do they compare?  What does that suggest about each model?  Do the same for accuracy?  Which do you think is more meaningful, the loss or the accuracy?

3) Suggest an improvement to one of the models (changing structure, learning rate, number of epochs, etc.) that you think will result in a better model.  Try it out below?  Did it improve the performance?